In [1]:
# Qpt

In [1]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

import numpy.testing as npt

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.data_analysis.projected_gradient_descent_backtracking import (
    ProjectedGradientDescentBacktracking,
    ProjectedGradientDescentBacktrackingOption,
)
from quara.data_analysis.weighted_probability_based_squared_error import (
    WeightedProbabilityBasedSquaredError,
    WeightedProbabilityBasedSquaredErrorOption,
)
from quara.data_analysis.weighted_relative_entropy import (
    WeightedRelativeEntropy,
    WeightedRelativeEntropyOption,
)
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.state import (
    State,
    get_x0_1q,
    get_x1_1q,
    get_y0_1q,
    get_y1_1q,
    get_z0_1q,
    get_z1_1q,
)
from quara.objects.povm import Povm, get_x_povm, get_y_povm, get_z_povm
from quara.objects.gate import (
    Gate,
    get_depolarizing_channel,
    get_x_rotation,
    get_amplitutde_damping_channel,
)
from quara.objects.qoperation import QOperation
from quara.protocol.qtomography.standard.standard_qpt import StandardQpt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import (
    LossMinimizationEstimator,
)
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

from quara.data_analysis.simulation import StandardQTomographySimulationSetting
from quara.settings import Settings

In [73]:
import quara.objects.state_typical as st
from quara.objects.operators import tensor_product

In [3]:
%load_ext autoreload
%autoreload 2

In [10]:
st.get_state_names()

['x0',
 'x1',
 'y0',
 'y1',
 'z0',
 'z1',
 'a',
 'bell_phi_plus',
 'bell_phi_minus',
 'bell_psi_plus',
 'bell_psi_minus',
 'z0_z0']

In [31]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

state = st.generate_state_from_state_name("y0", c_sys)

In [32]:
state_x0 = get_x0_1q(c_sys)
state_y0 = get_y0_1q(c_sys)
# |0><0|
state_z0 = get_z0_1q(c_sys)
# |1><1|
state_z1 = get_z1_1q(c_sys)

In [33]:
npt.assert_almost_equal(state.vec,state_y0.vec)
assert state.composite_system == state_y0.composite_system

In [30]:
id(state.composite_system.elemental_systems[0])

4651564432

In [29]:
id(state_y0.composite_system.elemental_systems[0])

4703539792

In [218]:
from quara.objects.matrix_basis import MatrixBasis, convert_vec, get_pauli_basis

In [51]:
from itertools import product

In [143]:
state_z0z1 = st.generate_state_from_state_name("z0_z1", c_sys)

state_z0z1.composite_system.elemental_systems[0].basis.basis

(array([[0.70710678+0.j, 0.        +0.j],
        [0.        +0.j, 0.70710678+0.j]]),
 array([[0.        +0.j, 0.70710678+0.j],
        [0.70710678+0.j, 0.        +0.j]]),
 array([[0.+0.j        , 0.-0.70710678j],
        [0.+0.70710678j, 0.+0.j        ]]),
 array([[ 0.70710678+0.j,  0.        +0.j],
        [ 0.        +0.j, -0.70710678+0.j]]))

In [275]:
# (1) to_density_matrix()
basis = get_normalized_pauli_basis()
e_sys0 = ElementalSystem(0, basis)
e_sys1 = ElementalSystem(1, basis)
c_sys = CompositeSystem([e_sys0, e_sys1])

state_z0z1 = st.generate_state_from_state_name("z0_z1", c_sys)
state_z1z0 = st.generate_state_from_state_name("z1_z0", c_sys)

from_mat = state_z0z1.to_density_matrix() - state_z1z0.to_density_matrix()
from_mat = (1 / np.sqrt(2)) * from_mat

basis = get_pauli_basis(2)
vec = calc_hermitian_matrix_expansion_coefficient_hermitian_basis(from_mat, basis)

State(vec=vec, c_sys=c_sys)

ValueError: the state is not physically correct.

In [278]:
# (2)
basis = get_normalized_pauli_basis()
e_sys = ElementalSystem(0, basis)
c_sys = CompositeSystem([e_sys])

state_z0 =  st.generate_state_from_state_name("z0", c_sys)
state_z1 = st.generate_state_from_state_name("z1", c_sys)

_vec = (1/np.sqrt(2)) * (state_z0z1.vec - state_z1z0.vec)
from_mat = _vec.reshape(4, 4)
print(from_mat)
basis = get_pauli_basis(2)
vec = calc_hermitian_matrix_expansion_coefficient_hermitian_basis(from_mat, basis)

# tensor_product(state_z0, state_z1).vec - tensor_product(state_z1, state_z0)

[[ 0.          0.          0.         -0.70710678]
 [ 0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]
 [ 0.70710678  0.          0.          0.        ]]


AssertionError: 

In [228]:
State(c_sys=c_sys, vec=vec)

ValueError: the state is not physically correct.

In [216]:
basis.dim

2

In [164]:
from quara.objects.matrix_basis import calc_hermitian_matrix_expansion_coefficient_hermitian_basis
from quara.utils.matrix_util import is_hermitian

In [118]:
basis = get_normalized_pauli_basis()

MatrixBasis(basis=[array([[0.70710678+0.j, 0.        +0.j],
       [0.        +0.j, 0.70710678+0.j]]), array([[0.        +0.j, 0.70710678+0.j],
       [0.70710678+0.j, 0.        +0.j]]), array([[0.+0.j        , 0.-0.70710678j],
       [0.+0.70710678j, 0.+0.j        ]]), array([[ 0.70710678+0.j,  0.        +0.j],
       [ 0.        +0.j, -0.70710678+0.j]])])

In [117]:
coeff_real = calc_hermitian_matrix_expansion_coefficient_hermitian_basis(from_mat, basis)

NameError: name 'from_mat' is not defined

In [213]:
print(get_z0_1q(c_sys).vec ==  st.generate_state_from_state_name("z0", c_sys).vec)
print(get_z1_1q(c_sys).vec ==  st.generate_state_from_state_name("z1", c_sys).vec)

[ True  True  True  True]
[ True  True  True  True]


In [243]:
from scipy.linalg import expm, sinm, cosm

In [244]:
# 1qubit

In [261]:
st.generate_state_from_state_name("x0", c_sys).to_density_matrix()

array([[0.5+0.j, 0.5+0.j],
       [0.5+0.j, 0.5+0.j]])

In [262]:
st.generate_state_from_state_name("x1", c_sys).to_density_matrix()

array([[ 0.5+0.j, -0.5+0.j],
       [-0.5+0.j,  0.5+0.j]])

In [263]:
st.generate_state_from_state_name("x1", c_sys).vec

array([ 0.70710678, -0.70710678,  0.        ,  0.        ])

In [259]:
st.generate_state_from_state_name("y0", c_sys).to_density_matrix()

array([[0.5+0.j , 0. -0.5j],
       [0. +0.5j, 0.5+0.j ]])

In [269]:
z0_state.to_density_matrix()

array([[1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j]])

In [ ]:
# 


In [286]:
# (1) 密度行列
basis = get_normalized_pauli_basis()
e_sys = ElementalSystem(0, basis)
c_sys = CompositeSystem([e_sys])

z0_state =  st.generate_state_from_state_name("z0", c_sys)
z1_state = st.generate_state_from_state_name("z1", c_sys)

z0_mat = z0_state.to_density_matrix()  # 密度行列  [[1.+0.j, 0.+0.j], [0.+0.j, 0.+0.j]]
z1_mat = z1_state.to_density_matrix()  # 密度行列 [[0.+0.j, 0.+0.j], [0.+0.j, 1.+0.j]]

mat = (1 / np.sqrt(2)) * (z0_mat + np.exp(1* np.pi / 4) * z1_mat)

print(mat) # [[0.70710678+0.j  0.+0.j ], [0.+0.j  0.5+0.5j]]

vec = calc_hermitian_matrix_expansion_coefficient_hermitian_basis(mat, basis)  # mutil.is_hermitian(from_mat) = False

# print(vec)
State(vec=vec, c_sys=c_sys)

[[0.70710678+0.j 0.        +0.j]
 [0.        +0.j 1.5508832 +0.j]]


ValueError: the state is not physically correct.

In [287]:
basis.shape

AttributeError: 'MatrixBasis' object has no attribute 'shape'

In [284]:
# (2) vec
basis = get_normalized_pauli_basis()
e_sys = ElementalSystem(0, basis)
c_sys = CompositeSystem([e_sys])

z0_state =  st.generate_state_from_state_name("z0", c_sys)
z1_state = st.generate_state_from_state_name("z1", c_sys)

z0_vec = z0_state.vec  # [0.70710678, 0.        , 0.        , 0.70710678]
z1_vec = z1_state.vec  # [ 0.70710678,  0.        ,  0.        , -0.70710678]

_vec = z0_state.vec + np.exp(np.pi / 4) * z1_state.vec
_vec = (1 / np.sqrt(2)) * _vec

mat = _vec.reshape(2, 2)  # [[ 1.59664003  0. ] [ 0. -0.59664003]]
print(mat)

vec = calc_hermitian_matrix_expansion_coefficient_hermitian_basis(mat, basis)

print(vec)

State(vec=vec, c_sys=c_sys)

[[ 1.59664003  0.        ]
 [ 0.         -0.59664003]]
[0.70710678 0.         0.         1.5508832 ]


ValueError: the state is not physically correct.

In [ ]:
_vec  = (np.array([1, 0, 0, 1]) + np.exp(np.pi / 4) * np.array([1, 0, 0, -1]))
_vec  =  (1 / np.sqrt(2)) * _vec

mat = _vec.reshape(2, 2)

print(mat)

vec = calc_hermitian_matrix_expansion_coefficient_hermitian_basis(mat, basis)